In [21]:
# Anipose_triangulation
from anipose.triangulate import process_session
from anipose.triangulate import triangulate
import toml
import os
from glob import glob
from collections import defaultdict
from anipose.common import find_calibration_folder, get_video_name, natural_keys, get_cam_name
from aniposelib.cameras import CameraGroup

In [2]:
config_file = '/home/yiting/Documents/Anipose_projects/config_2cam.toml'
session_path = '/home/yiting/Documents/Anipose_projects/Anipose_240624/231121-102936'
# p3d_file = os.path.join(session_path,'vid_231121-102936_.csv')
# p3d_video = os.path.join(session_path,'vid_231121-102936_.mp4')
# calibration_file = os.path.join(session_path,'calibration','calibration.toml')
# videos_raw_folder = os.path.join(session_path,'videos_raw')
# output_video = os.path.join(session_path,'label_combined.mp4')

In [5]:
# Load the TOML file
config = toml.load(config_file)

# # Loading calibration file
# cgroup = CameraGroup.load(calibration_file)
# cam_names = ['A', 'B']

# # Load offsets_dict
# offsets_dict = load_offsets_dict(config, cam_names, videos_raw_folder)

In [6]:
process_session(config, session_path)

/home/yiting/Documents/Anipose_projects/Anipose_240624/231121-102936/pose_3d/vid_231121-102936_.csv


TypeError: can only concatenate str (not "int") to str

In [ ]:
triangulate(config,
                calib_folder, video_folder, pose_folder,
                fname_dict, output_fname)

In [22]:
cam_names = sorted(fname_dict.keys())

calib_fname = os.path.join(calib_folder, 'calibration.toml')
cgroup = CameraGroup.load(calib_fname)

offsets_dict = load_offsets_dict(config, cam_names, video_folder)

out = load_pose2d_fnames(fname_dict, offsets_dict, cam_names)
all_points_raw = out['points']
all_scores = out['scores']
bodyparts = out['bodyparts']

cgroup = cgroup.subset_cameras_names(cam_names)

n_cams, n_frames, n_joints, _ = all_points_raw.shape

bad = all_scores < config['triangulation']['score_threshold']
all_points_raw[bad] = np.nan

# if config['triangulation']['optim']:
#     constraints = load_constraints(config, bodyparts)
#     constraints_weak = load_constraints(config, bodyparts, 'constraints_weak')

#     points_2d = all_points_raw
#     scores_2d = all_scores

#     points_shaped = points_2d.reshape(n_cams, n_frames*n_joints, 2)
#     if config['triangulation']['ransac']:
#         points_3d_init, _, _, _ = cgroup.triangulate_ransac(points_shaped, progress=True)
#     else:
#         points_3d_init = cgroup.triangulate(points_shaped, progress=True)
#     points_3d_init = points_3d_init.reshape((n_frames, n_joints, 3))

#     c = np.isfinite(points_3d_init[:, :, 0])
#     if np.sum(c) < 20:
#         print("warning: not enough 3D points to run optimization")
#         points_3d = points_3d_init
#     else:
#         points_3d = cgroup.optim_points(
#             points_2d, points_3d_init,
#             constraints=constraints,
#             constraints_weak=constraints_weak,
#             # scores=scores_2d,
#             scale_smooth=config['triangulation']['scale_smooth'],
#             scale_length=config['triangulation']['scale_length'],
#             scale_length_weak=config['triangulation']['scale_length_weak'],
#             n_deriv_smooth=config['triangulation']['n_deriv_smooth'],
#             reproj_error_threshold=config['triangulation']['reproj_error_threshold'],
#             verbose=True)

#     points_2d_flat = points_2d.reshape(n_cams, -1, 2)
#     points_3d_flat = points_3d.reshape(-1, 3)

#     errors = cgroup.reprojection_error(
#         points_3d_flat, points_2d_flat, mean=True)
#     good_points = ~np.isnan(all_points_raw[:, :, :, 0])
#     num_cams = np.sum(good_points, axis=0).astype('float')

#     all_points_3d = points_3d
#     all_errors = errors.reshape(n_frames, n_joints)

#     all_scores[~good_points] = 2
#     scores_3d = np.min(all_scores, axis=0)

#     scores_3d[num_cams < 1] = np.nan
#     all_errors[num_cams < 1] = np.nan

# else:
#     points_2d = all_points_raw.reshape(n_cams, n_frames*n_joints, 2)
#     if config['triangulation']['ransac']:
#         points_3d, picked, p2ds, errors = cgroup.triangulate_ransac(
#             points_2d, min_cams=3, progress=True)

#         all_points_picked = p2ds.reshape(n_cams, n_frames, n_joints, 2)
#         good_points = ~np.isnan(all_points_picked[:, :, :, 0])

#         num_cams = np.sum(np.sum(picked, axis=0), axis=1)\
#                         .reshape(n_frames, n_joints)\
#                         .astype('float')
#     else:
#         points_3d = cgroup.triangulate(points_2d, progress=True)
#         errors = cgroup.reprojection_error(points_3d, points_2d, mean=True)
#         good_points = ~np.isnan(all_points_raw[:, :, :, 0])
#         num_cams = np.sum(good_points, axis=0).astype('float')

#     all_points_3d = points_3d.reshape(n_frames, n_joints, 3)
#     all_errors = errors.reshape(n_frames, n_joints)

#     all_scores[~good_points] = 2
#     scores_3d = np.min(all_scores, axis=0)

#     scores_3d[num_cams < 2] = np.nan
#     all_errors[num_cams < 2] = np.nan
#     num_cams[num_cams < 2] = np.nan

# if 'reference_point' in config['triangulation'] and 'axes' in config['triangulation']:
#     all_points_3d_adj, M, center = correct_coordinate_frame(config, all_points_3d, bodyparts)
# else:
#     all_points_3d_adj = all_points_3d
#     M = np.identity(3)
#     center = np.zeros(3)

# dout = pd.DataFrame()
# for bp_num, bp in enumerate(bodyparts):
#     for ax_num, axis in enumerate(['x','y','z']):
#         dout[bp + '_' + axis] = all_points_3d_adj[:, bp_num, ax_num]
#     dout[bp + '_error'] = all_errors[:, bp_num]
#     dout[bp + '_ncams'] = num_cams[:, bp_num]
#     dout[bp + '_score'] = scores_3d[:, bp_num]

# for i in range(3):
#     for j in range(3):
#         dout['M_{}{}'.format(i, j)] = M[i, j]

# for i in range(3):
#     dout['center_{}'.format(i)] = center[i]

# dout['fnum'] = np.arange(n_frames)

# dout.to_csv(output_fname, index=False)

NameError: name 'load_offsets_dict' is not defined

In [12]:
pipeline_videos_raw = config['pipeline']['videos_raw']
pipeline_calibration_results = config['pipeline']['calibration_results']
pipeline_pose = config['pipeline']['pose_2d']
pipeline_pose_filter = config['pipeline']['pose_2d_filter']
pipeline_3d = config['pipeline']['pose_3d']

calibration_path = find_calibration_folder(config, session_path)
# if calibration_path is None:
#     return

if config['filter']['enabled']:
    pose_folder = os.path.join(session_path, pipeline_pose_filter)
else:
    pose_folder = os.path.join(session_path, pipeline_pose)

calib_folder = os.path.join(calibration_path, pipeline_calibration_results)
video_folder = os.path.join(session_path, pipeline_videos_raw)
output_folder = os.path.join(session_path, pipeline_3d)

pose_files = glob(os.path.join(pose_folder, '*.h5'))

cam_videos = defaultdict(list)

for pf in pose_files:
    name = get_video_name(config, pf)
    cam_videos[name].append(pf)

vid_names = cam_videos.keys()
vid_names = sorted(vid_names, key=natural_keys)

if len(vid_names) > 0:
    os.makedirs(output_folder, exist_ok=True)



In [18]:
name = vid_names[0]
fnames = cam_videos[name]
cam_names = [get_cam_name(config, f) for f in fnames]
fname_dict = dict(zip(cam_names, fnames))

output_fname = os.path.join(output_folder, name + '.csv')

print(output_fname)


/home/yiting/Documents/Anipose_projects/Anipose_240624/231121-102936/pose_3d/vid_231121-102936_.csv


In [19]:
triangulate(config,calib_folder, video_folder, pose_folder,fname_dict, output_fname)

TypeError: can only concatenate str (not "int") to str

In [ ]:
def process_session(config, session_path):
    pipeline_videos_raw = config['pipeline']['videos_raw']
    pipeline_calibration_results = config['pipeline']['calibration_results']
    pipeline_pose = config['pipeline']['pose_2d']
    pipeline_pose_filter = config['pipeline']['pose_2d_filter']
    pipeline_3d = config['pipeline']['pose_3d']

    calibration_path = find_calibration_folder(config, session_path)
    if calibration_path is None:
        return

    if config['filter']['enabled']:
        pose_folder = os.path.join(session_path, pipeline_pose_filter)
    else:
        pose_folder = os.path.join(session_path, pipeline_pose)

    calib_folder = os.path.join(calibration_path, pipeline_calibration_results)
    video_folder = os.path.join(session_path, pipeline_videos_raw)
    output_folder = os.path.join(session_path, pipeline_3d)

    pose_files = glob(os.path.join(pose_folder, '*.h5'))

    cam_videos = defaultdict(list)

    for pf in pose_files:
        name = get_video_name(config, pf)
        cam_videos[name].append(pf)

    vid_names = cam_videos.keys()
    vid_names = sorted(vid_names, key=natural_keys)

    if len(vid_names) > 0:
        os.makedirs(output_folder, exist_ok=True)

    for name in vid_names:
        fnames = cam_videos[name]
        cam_names = [get_cam_name(config, f) for f in fnames]
        fname_dict = dict(zip(cam_names, fnames))

        output_fname = os.path.join(output_folder, name + '.csv')

        print(output_fname)
        
        if os.path.exists(output_fname):
            continue


        try:
            triangulate(config,
                        calib_folder, video_folder, pose_folder,
                        fname_dict, output_fname)
        except ValueError:
            import traceback, sys
            traceback.print_exc(file=sys.stdout)
            

In [ ]:
triangulate(config,
                calib_folder, video_folder, pose_folder,
                fname_dict, output_fname)